In [1]:
import pandas as pd
import json

In [121]:
with open('../data/yelp/ANS_polar-stars_2rest100.json') as f:
    gold = json.load(f)

gold[:2]

[{'sent_id': 'LxMrMDZvwNL7btnuqKYm8g:0',
  'text': 'I agree with the previous review.',
  'opinions': [{'Source': [['I'], ['0:1']],
    'Target': [['review.'], ['26:33']],
    'Polar_expression': [['agree'], ['2:7']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'}],
  'business_id': '_ab50qdWOk0DdB6XOrBitw',
  'review_id': 'LxMrMDZvwNL7btnuqKYm8g',
  'stars': 1.0,
  'polar_ans': True},
 {'sent_id': 'LxMrMDZvwNL7btnuqKYm8g:1',
  'text': 'You can definitely find better restaurants in New Orleans.',
  'opinions': [{'Source': [[], []],
    'Target': [['restaurants'], ['31:42']],
    'Polar_expression': [['better'], ['24:30']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'}],
  'business_id': '_ab50qdWOk0DdB6XOrBitw',
  'review_id': 'LxMrMDZvwNL7btnuqKYm8g',
  'stars': 1.0,
  'polar_ans': True}]

In [122]:
# rest1: '_ab50qdWOk0DdB6XOrBitw'
# rest2: 'ac1AeYqs8Z4_e2X5M3if2A' 

# split into 2 rests
gold1, gold2 = [], []

for i in gold:
    if i['business_id'] == '_ab50qdWOk0DdB6XOrBitw':
        gold1.append(i)
    else:
        gold2.append(i)

print(len(gold), len(gold1), len(gold2))

710 347 363


In [123]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /share/home/qwe9887476/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# define stopwords
stopword = stopwords.words('english') 
stopwords

<WordListCorpusReader in '/share/home/qwe9887476/nltk_data/corpora/stopwords'>

In [6]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [124]:
# build a list of all target
target_list1, target_list2 = [], []

for goldlist in [gold1, gold2]:

    L = []

    for sent in range(len(goldlist)):
        opi = goldlist[sent]['opinions']

        if len(opi) > 0: # non empty opinion
            for i in range(len(opi)):
                goldlist[sent]['opinions'][i]['Target_key'] = [] # add new key 'Target_key' containing tokens of 'Target'
                # print(opi[i])

                if len(opi[i]['Target'][0]) > 0: # pred_target exists
                    for tgt in opi[i]['Target'][0]:
                        # remove all punctuation
                        tgt = tgt.translate(str.maketrans('', '', string.punctuation))

                        # split the target text
                        tgt_tokens = nltk.word_tokenize(tgt.lower())

                        # remove stopwords
                        tgt_tokens_nostop = [k for k in tgt_tokens if k not in stopword]
                        # print(tgt_tokens_nostop)

                        # add all words to the list
                        if len(tgt_tokens_nostop) > 0:
                            for t in tgt_tokens_nostop:
                                L.append(t)
                                goldlist[sent]['opinions'][i]['Target_key'].append(t)

    if goldlist == gold1:
        target_list1 = L
    else:
        target_list2 = L

In [125]:
gold1[:5]

[{'sent_id': 'LxMrMDZvwNL7btnuqKYm8g:0',
  'text': 'I agree with the previous review.',
  'opinions': [{'Source': [['I'], ['0:1']],
    'Target': [['review.'], ['26:33']],
    'Polar_expression': [['agree'], ['2:7']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Target_key': ['review']}],
  'business_id': '_ab50qdWOk0DdB6XOrBitw',
  'review_id': 'LxMrMDZvwNL7btnuqKYm8g',
  'stars': 1.0,
  'polar_ans': True},
 {'sent_id': 'LxMrMDZvwNL7btnuqKYm8g:1',
  'text': 'You can definitely find better restaurants in New Orleans.',
  'opinions': [{'Source': [[], []],
    'Target': [['restaurants'], ['31:42']],
    'Polar_expression': [['better'], ['24:30']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Target_key': ['restaurants']}],
  'business_id': '_ab50qdWOk0DdB6XOrBitw',
  'review_id': 'LxMrMDZvwNL7btnuqKYm8g',
  'stars': 1.0,
  'polar_ans': True},
 {'sent_id': 'LxMrMDZvwNL7btnuqKYm8g:2',
  'text': 'Buyer beware: the food does not live up to its hype.',
  'opi

In [126]:
# check different opinions' polarity of one sent when length of opinions > 1

for GOLD in [gold1, gold2]:
    r = 1
    print(f'------rest{r}------')
    r += 1
    
    diff = 0
    same = 0
    total = 0
    for i in GOLD:
        if len(i['opinions']) > 0:
            total += 1
        if len(i['opinions']) > 1:
            flag = 1

            for j in range(len(i['opinions'])):
                if (j != 0) and (i['opinions'][j]['Polarity'] != i['opinions'][j-1]['Polarity']):
                    flag = 0 # different polar
                    # print(j['Polarity'])
                    break
            
            if flag == 0:
                diff += 1
            else:
                same += 1

    print(f'total(opinion > 0): {total}\ndiff polar(opinion > 1): {diff}\nsame polar(opinion > 1): {same}')

------rest1------
total(opinion > 0): 214
diff polar(opinion > 1): 10
same polar(opinion > 1): 48
------rest1------
total(opinion > 0): 219
diff polar(opinion > 1): 7
same polar(opinion > 1): 47


In [127]:
# dict of word frequency
counts1, counts2 = {}, {}

for word in target_list1:
    counts1[word] = counts1.get(word,0) + 1

for word in target_list2:
    counts2[word] = counts2.get(word,0) + 1

print(counts1)
print('------------------')
print(counts2)

{'review': 2, 'restaurants': 1, 'food': 21, 'atmosphere': 5, 'bar': 2, 'experience': 6, 'shucker': 1, 'seafood': 4, 'gumbo': 8, 'oysters': 31, 'local': 2, 'headache': 1, 'wet': 1, 'toes': 1, 'hair': 1, 'fast': 1, 'service': 11, 'wait': 5, 'shrimp': 3, 'medley': 2, 'platter': 1, 'beans': 2, 'server': 3, 'oyster': 6, 'fan': 1, 'vibes': 1, 'smell': 1, 'well': 1, 'charbroiled': 2, 'amount': 2, 'plates': 1, 'rice': 1, 'meal': 1, 'acme': 2, 'house': 2, 'street': 1, 'parking': 1, 'spots': 1, 'timing': 1, 'host': 2, 'servers': 3, 'everything': 6, 'place': 15, 'garlic': 1, 'people': 2, 'environment': 1, 'quality': 1, 'cheese': 1, 'mkt': 1, 'price': 3, 'jambalaya': 4, 'shooter': 1, 'line': 5, 'charred': 1, 'item': 1, 'fish': 3, 'texture': 1, 'cravings': 1, 'crab': 4, 'cakes': 1, 'lots': 1, 'crabmeat': 1, 'long': 1, 'lines': 1, 'fried': 2, 'maker': 1, 'poboy': 1, 'chargrilled': 5, 'spot': 1, 'staff': 8, 'rest': 1, 'po': 3, 'boy': 2, 'etouffé': 1, 'options': 2, 'waitress': 1, 'claws': 1, 'suggesti

In [128]:
print(len(list(counts1.keys())), len(list(counts2.keys())))

103 125


In [129]:
# combine the word frquency of singular and plural noun
# leave singular noun only

for COUNTS in [counts1, counts2]:
   
    for k1 in list(COUNTS.keys()):
        for k2 in list(COUNTS.keys()):
            if (len(k1) < len(k2)) and (k2 == k1 + 's'):
                # print(k1, v1, k2, v2)
                COUNTS[k1] += COUNTS[k2]
                del COUNTS[k2]

print(counts1)
print('------------------')
print(counts2)    

{'review': 3, 'food': 21, 'atmosphere': 5, 'bar': 2, 'experience': 6, 'shucker': 1, 'seafood': 4, 'gumbo': 8, 'local': 2, 'headache': 1, 'wet': 1, 'toes': 1, 'hair': 1, 'fast': 1, 'service': 11, 'wait': 5, 'shrimp': 3, 'medley': 2, 'platter': 1, 'beans': 2, 'server': 6, 'oyster': 37, 'fan': 1, 'vibes': 1, 'smell': 1, 'well': 1, 'charbroiled': 2, 'amount': 2, 'plates': 1, 'rice': 1, 'meal': 1, 'acme': 2, 'house': 2, 'street': 1, 'parking': 1, 'timing': 1, 'host': 2, 'everything': 6, 'place': 15, 'garlic': 1, 'people': 2, 'environment': 1, 'quality': 1, 'cheese': 1, 'mkt': 1, 'price': 4, 'jambalaya': 4, 'shooter': 1, 'line': 6, 'charred': 1, 'item': 1, 'fish': 3, 'texture': 1, 'cravings': 1, 'crab': 4, 'cakes': 1, 'lots': 1, 'crabmeat': 1, 'long': 1, 'fried': 2, 'maker': 1, 'poboy': 1, 'chargrilled': 5, 'spot': 2, 'staff': 8, 'rest': 1, 'po': 3, 'boy': 3, 'etouffé': 1, 'options': 2, 'waitress': 1, 'claws': 1, 'suggestion': 1, 'homes': 1, 'restaurant': 3, 'sauce': 3, 'bussers': 1, 'corn':

In [130]:
print(len(list(counts1.keys())), len(list(counts2.keys())))

95 117


In [131]:
# Target Word Frequency Top 10

print('Target Word Frequency Top 10')

r = 1
for COUNTS in [counts1, counts2]:

    print(f'------rest{r}------')
    r += 1

    items = list(COUNTS.items())
    items.sort(key=lambda x:x[1], reverse=True)

    for j in range(10):
        word, count = items[j]
        print("{0:<10}{1:>5}".format(word, count))

Target Word Frequency Top 10
------rest1------
oyster       37
food         21
place        15
service      11
gumbo         8
staff         8
experience    6
server        6
everything    6
line          6
------rest2------
food         26
place        13
service      13
staff        10
restaurant    9
waiter        6
drinks        6
crawfish      6
seafood       5
experience    5


In [132]:
# singular and plural form are consideres the same in 'Target_key'

for GLD in [gold1, gold2]:

    if GLD == gold1:
        KEY = list(counts1.keys())
    else:
        KEY = list(counts2.keys())
        
    for i in range(len(GLD)):
        if len(GLD[i]['opinions']) > 0: # non empty opinion
            
            for j in range(len(GLD[i]['opinions'])):
                if (len(GLD[i]['opinions'][j]['Target_key'])) > 0:

                    for t in range(len(GLD[i]['opinions'][j]['Target_key'])):
                        word = GLD[i]['opinions'][j]['Target_key'][t]
                        if (word.endswith('s')) and (word not in KEY) :
                            # print(word)
                            GLD[i]['opinions'][j]['Target_key'][t] = word[:-1] # remove 's' from plural words

gold1[:3]


[{'sent_id': 'LxMrMDZvwNL7btnuqKYm8g:0',
  'text': 'I agree with the previous review.',
  'opinions': [{'Source': [['I'], ['0:1']],
    'Target': [['review.'], ['26:33']],
    'Polar_expression': [['agree'], ['2:7']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Target_key': ['review']}],
  'business_id': '_ab50qdWOk0DdB6XOrBitw',
  'review_id': 'LxMrMDZvwNL7btnuqKYm8g',
  'stars': 1.0,
  'polar_ans': True},
 {'sent_id': 'LxMrMDZvwNL7btnuqKYm8g:1',
  'text': 'You can definitely find better restaurants in New Orleans.',
  'opinions': [{'Source': [[], []],
    'Target': [['restaurants'], ['31:42']],
    'Polar_expression': [['better'], ['24:30']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Target_key': ['restaurant']}],
  'business_id': '_ab50qdWOk0DdB6XOrBitw',
  'review_id': 'LxMrMDZvwNL7btnuqKYm8g',
  'stars': 1.0,
  'polar_ans': True},
 {'sent_id': 'LxMrMDZvwNL7btnuqKYm8g:2',
  'text': 'Buyer beware: the food does not live up to its hype.',
  'opin

In [112]:
import pandas as pd

In [146]:
# rest1's statistic results by target key
# df column: key | total_sent| TRUE_polar_ans | FALSE_polar_ans | accuracy | TP | TN | FP | FN | total_opinion | Positive | Negative
# important: TP, TN, FP and FN depend on first opinion's polarity only!!!

key1 = list(counts1.keys())
total_sent1, TRUE_polar_ans1, FALSE_polar_ans1, accur1 = [], [], [], []
total_opinion1, Pos1, Neg1, TP1, FP1 = [], [], [], [], []
TN1, FN1 = [], []

for k in key1:
    sent, t, f = 0, 0, 0
    opinion, pos, neg, tp, fp = 0, 0, 0, 0, 0

    for g in gold1:
        sign = False
        if (len(g['opinions']) > 0): # non empty 'opinions'
            for o in g['opinions']: # every opinion in 'opinions'
                if (len(o['Target_key']) > 0): # non empty 'Target_key'
                    for tgk in o['Target_key']: # every word in 'Target_key'

                        if tgk == k: 
                            sign = True
                            opinion += 1
                            if o['Polarity'] == 'Positive':
                                 pos += 1
                            else: 
                                neg += 1

            if sign == True:
                sent += 1
                if g['polar_ans']: # gold ans: True
                    t += 1
                    if g['opinions'][0]['Polarity'] == 'Positive': 
                        tp += 1
                else: # gold ans: False
                    f += 1
                    if g['opinions'][0]['Polarity'] == 'Positive': 
                        fp += 1
    
    total_sent1.append(sent)
    TRUE_polar_ans1.append(t)
    FALSE_polar_ans1.append(f)
    accur1.append("%.2f%%" % (100 * t/sent))
    total_opinion1.append(opinion)
    Pos1.append(pos)
    Neg1.append(neg)
    TP1.append(tp)
    FP1.append(fp)
    TN1.append(t-tp)
    FN1.append(f-fp)



In [147]:
# rest1's stat

df_stat1 = pd.DataFrame({'key': key1, 'total_sent': total_sent1, 'TRUE_polar_ans': TRUE_polar_ans1, 'FALSE_polar_ans': FALSE_polar_ans1,
                        'accuracy': accur1, 'TP': TP1, 'TN':  TN1, 'FP': FP1, 'FN': FN1,
                        'total_opinion': total_opinion1, 'Positive': Pos1, 'Negative': Neg1})

df_stat1 = df_stat1.sort_values(['total_sent', 'total_opinion'], ascending=False)
df_stat1[df_stat1.total_sent > 2]
# df_stat1

,key,total_sent,TRUE_polar_ans,FALSE_polar_ans,accuracy,TP,TN,FP,FN,total_opinion,Positive,Negative
21,oyster,32,27,5,84.38%,24,3,1,4,37,31,6
1,food,18,14,4,77.78%,11,3,2,2,21,13,8
38,place,15,13,2,86.67%,12,1,1,1,15,12,3
14,service,9,8,1,88.89%,8,0,1,0,11,11,0
7,gumbo,8,6,2,75.00%,4,2,0,2,8,5,3
64,staff,5,3,2,60.00%,3,0,1,1,8,6,2
4,experience,5,4,1,80.00%,2,2,0,1,6,3,3
20,server,5,5,0,100.00%,4,1,0,0,6,5,1
48,line,5,3,2,60.00%,1,2,0,2,6,1,5
2,atmosphere,5,5,0,100.00%,3,2,0,0,5,3,2


In [148]:
# rest2's statistic results by target key
# df column: key | total_sent| TRUE_polar_ans | FALSE_polar_ans | accuracy | TP | TN | FP | FN | total_opinion | Positive | Negative
# important: TP, TN, FP and FN depend on first opinion's polarity only!!!

key2 = list(counts2.keys())
total_sent2, TRUE_polar_ans2, FALSE_polar_ans2, accur2 = [], [], [], []
total_opinion2, Pos2, Neg2, TP2, FP2 = [], [], [], [], []
TN2, FN2 = [], []

for k in key2:
    sent, t, f = 0, 0, 0
    opinion, pos, neg, tp, fp = 0, 0, 0, 0, 0

    for g in gold2:
        sign = False
        if (len(g['opinions']) > 0): # non empty 'opinions'
            for o in g['opinions']: # every opinion in 'opinions'
                if (len(o['Target_key']) > 0): # non empty 'Target_key'
                    for tgk in o['Target_key']: # every word in 'Target_key'

                        if tgk == k: 
                            sign = True
                            opinion += 1
                            if o['Polarity'] == 'Positive':
                                pos += 1
                            else: 
                                neg += 1

            if sign == True:
                sent += 1
                if g['polar_ans']: # gold ans: True
                    t += 1
                    if g['opinions'][0]['Polarity'] == 'Positive': 
                        tp += 1
                else: # gold ans: False
                    f += 1
                    if g['opinions'][0]['Polarity'] == 'Positive': 
                        fp += 1

    total_sent2.append(sent)
    TRUE_polar_ans2.append(t)
    FALSE_polar_ans2.append(f)
    accur2.append("%.2f%%" % (100 * t/sent))
    total_opinion2.append(opinion)
    Pos2.append(pos)
    Neg2.append(neg)
    TP2.append(tp)
    FP2.append(fp)
    TN2.append(t-tp)
    FN2.append(f-fp)

In [149]:
df_stat2 = pd.DataFrame({'key': key2, 'total_sent': total_sent2, 'TRUE_polar_ans': TRUE_polar_ans2, 'FALSE_polar_ans': FALSE_polar_ans2,
                        'accuracy': accur2, 'TP': TP2, 'TN':  TN2, 'FP': FP2, 'FN': FN2,
                        'total_opinion': total_opinion2, 'Positive': Pos2, 'Negative': Neg2})

df_stat2 = df_stat2.sort_values(['total_sent', 'total_opinion'], ascending=False)
# df_stat2
df_stat2[df_stat2.total_sent > 2]


,key,total_sent,TRUE_polar_ans,FALSE_polar_ans,accuracy,TP,TN,FP,FN,total_opinion,Positive,Negative
6,food,22,21,1,95.45%,18,3,1,0,26,24,2
0,place,12,10,2,83.33%,10,0,1,1,13,12,1
5,service,11,10,1,90.91%,8,2,0,1,13,10,3
43,restaurant,8,7,1,87.50%,6,1,1,0,9,8,1
8,staff,6,6,0,100.00%,6,0,0,0,10,10,0
12,waiter,5,4,1,80.00%,4,0,0,1,6,5,1
15,drinks,5,4,1,80.00%,3,1,1,0,6,6,0
16,seafood,5,5,0,100.00%,4,1,0,0,5,4,1
33,experience,5,4,1,80.00%,3,1,1,0,5,4,1
67,crawfish,4,3,1,75.00%,3,0,1,0,6,6,0


In [152]:
# total accuracy

sum_sent1 = sum(total_sent1)
sum_sent2 = sum(total_sent2)

sum_T1 = sum(TRUE_polar_ans1)
sum_T2 = sum(TRUE_polar_ans2)

print('Accuracy by target key (total_sent >= 1):')
print('rest1: ' + str(sum_T1) + '/' + str(sum_sent1) + ' = ' "%.2f%%" % (100 * sum_T1/sum_sent1))
print('rest2: ' + str(sum_T2) + '/' + str(sum_sent2) + ' = ' "%.2f%%" % (100 * sum_T2/sum_sent2))
print('total: ' + str(sum_T1+sum_T2) + '/' + str(sum_sent1+sum_sent2) + ' = ' "%.2f%%" % (100 * (sum_T1+sum_T2)/(sum_sent1+sum_sent2)))

print('-----------------------')

print('Accuracy by target key (total_sent >= 5):')
print('rest1: ' + str(96) + '/' + str(117) + ' = ' "%.2f%%" % (100 * 96/117))
print('rest2: ' + str(71) + '/' + str(79) + ' = ' "%.2f%%" % (100 * 71/79))
print('total: ' + str(96+71) + '/' + str(117+79) + ' = ' "%.2f%%" % (100 * (96+71)/(117+79)))

Accuracy by target key (total_sent >= 1):
rest1: 198/234 = 84.62%
rest2: 192/229 = 83.84%
total: 390/463 = 84.23%
-----------------------
Accuracy by target key (total_sent >= 5):
rest1: 96/117 = 82.05%
rest2: 71/79 = 89.87%
total: 167/196 = 85.20%


In [138]:
# df to csv
df_stat1.to_csv('../data/yelp/STAT_polar-major_rest1.csv', index=False)
df_stat2.to_csv('../data/yelp/STAT_polar-major_rest2.csv', index=False)

In [139]:
# combine gold1 and gold2
gold_mix = []

for i in gold1:
    gold_mix.append(i)

for j in gold2:
    gold_mix.append(j)

In [140]:
# output json 
with open('../data/yelp/ANS_polar-major_2rest100.json', 'w') as outfile:
    json.dump(gold_mix, outfile, indent = 4, ensure_ascii=False)

### If we only count accuracy when length of opinion is 1

In [153]:
key1 = list(counts1.keys())
total_sent1, TRUE_polar_ans1, FALSE_polar_ans1, accur1 = [], [], [], []
total_opinion1, Pos1, Neg1, TP1, FP1 = [], [], [], [], []
TN1, FN1 = [], []

for k in key1:
    sent, t, f = 0, 0, 0
    opinion, pos, neg, tp, fp = 0, 0, 0, 0, 0

    for g in gold1:
        sign = False
        if (len(g['opinions']) == 1): # len(['opinions']) = 1
            for o in g['opinions']: # every opinion in 'opinions'
                if (len(o['Target_key']) > 0): # non empty 'Target_key'
                    for tgk in o['Target_key']: # every word in 'Target_key'

                        if tgk == k: 
                            sign = True
                            opinion += 1
                            if o['Polarity'] == 'Positive':
                                 pos += 1
                            else: 
                                neg += 1

            if sign == True:
                sent += 1
                if g['polar_ans']: # gold ans: True
                    t += 1
                    if g['opinions'][0]['Polarity'] == 'Positive': 
                        tp += 1
                else: # gold ans: False
                    f += 1
                    if g['opinions'][0]['Polarity'] == 'Positive': 
                        fp += 1
    
    total_sent1.append(sent)
    TRUE_polar_ans1.append(t)
    FALSE_polar_ans1.append(f)

    if sent > 0:
        accur1.append("%.2f%%" % (100 * t/sent))
    else:
        accur1.append("-")

    total_opinion1.append(opinion)
    Pos1.append(pos)
    Neg1.append(neg)
    TP1.append(tp)
    FP1.append(fp)
    TN1.append(t-tp)
    FN1.append(f-fp)

In [154]:
# rest1's stat

df_stat1 = pd.DataFrame({'key': key1, 'total_sent': total_sent1, 'TRUE_polar_ans': TRUE_polar_ans1, 'FALSE_polar_ans': FALSE_polar_ans1,
                        'accuracy': accur1, 'TP': TP1, 'TN':  TN1, 'FP': FP1, 'FN': FN1,
                        'total_opinion': total_opinion1, 'Positive': Pos1, 'Negative': Neg1})

df_stat1 = df_stat1.sort_values(['total_sent', 'total_opinion'], ascending=False)
df_stat1[df_stat1.total_sent > 2]
# df_stat1

,key,total_sent,TRUE_polar_ans,FALSE_polar_ans,accuracy,TP,TN,FP,FN,total_opinion,Positive,Negative
21,oyster,23,18,5,78.26%,16,2,1,4,23,17,6
38,place,11,9,2,81.82%,8,1,1,1,11,9,2
1,food,8,6,2,75.00%,4,2,1,1,8,5,3
7,gumbo,5,3,2,60.00%,2,1,0,2,5,2,3
62,chargrilled,5,5,0,100.00%,5,0,0,0,5,5,0
2,atmosphere,4,4,0,100.00%,3,1,0,0,4,3,1
4,experience,4,4,0,100.00%,2,2,0,0,4,2,2
15,wait,4,2,2,50.00%,1,1,0,2,4,1,3
0,review,3,3,0,100.00%,2,1,0,0,3,2,1
14,service,3,2,1,66.67%,2,0,1,0,3,3,0


In [155]:
# rest2's statistic results by target key
# df column: key | total_sent| TRUE_polar_ans | FALSE_polar_ans | accuracy | TP | TN | FP | FN | total_opinion | Positive | Negative
# important: TP, TN, FP and FN depend on first opinion's polarity only!!!

key2 = list(counts2.keys())
total_sent2, TRUE_polar_ans2, FALSE_polar_ans2, accur2 = [], [], [], []
total_opinion2, Pos2, Neg2, TP2, FP2 = [], [], [], [], []
TN2, FN2 = [], []

for k in key2:
    sent, t, f = 0, 0, 0
    opinion, pos, neg, tp, fp = 0, 0, 0, 0, 0

    for g in gold2:
        sign = False
        if (len(g['opinions']) == 1): # non empty 'opinions'
            for o in g['opinions']: # every opinion in 'opinions'
                if (len(o['Target_key']) > 0): # non empty 'Target_key'
                    for tgk in o['Target_key']: # every word in 'Target_key'

                        if tgk == k: 
                            sign = True
                            opinion += 1
                            if o['Polarity'] == 'Positive':
                                pos += 1
                            else: 
                                neg += 1

            if sign == True:
                sent += 1
                if g['polar_ans']: # gold ans: True
                    t += 1
                    if g['opinions'][0]['Polarity'] == 'Positive': 
                        tp += 1
                else: # gold ans: False
                    f += 1
                    if g['opinions'][0]['Polarity'] == 'Positive': 
                        fp += 1

    total_sent2.append(sent)
    TRUE_polar_ans2.append(t)
    FALSE_polar_ans2.append(f)

    if sent > 0:
        accur2.append("%.2f%%" % (100 * t/sent))
    else:
        accur2.append("-")

    total_opinion2.append(opinion)
    Pos2.append(pos)
    Neg2.append(neg)
    TP2.append(tp)
    FP2.append(fp)
    TN2.append(t-tp)
    FN2.append(f-fp)

In [156]:
df_stat2 = pd.DataFrame({'key': key2, 'total_sent': total_sent2, 'TRUE_polar_ans': TRUE_polar_ans2, 'FALSE_polar_ans': FALSE_polar_ans2,
                        'accuracy': accur2, 'TP': TP2, 'TN':  TN2, 'FP': FP2, 'FN': FN2,
                        'total_opinion': total_opinion2, 'Positive': Pos2, 'Negative': Neg2})

df_stat2 = df_stat2.sort_values(['total_sent', 'total_opinion'], ascending=False)
# df_stat2
df_stat2[df_stat2.total_sent > 2]

,key,total_sent,TRUE_polar_ans,FALSE_polar_ans,accuracy,TP,TN,FP,FN,total_opinion,Positive,Negative
0,place,10,9,1,90.00%,9,0,0,1,10,9,1
6,food,10,10,0,100.00%,8,2,0,0,10,8,2
5,service,7,7,0,100.00%,5,2,0,0,7,5,2
33,experience,5,4,1,80.00%,3,1,1,0,5,4,1
43,restaurant,5,4,1,80.00%,3,1,1,0,5,4,1
2,taste,3,3,0,100.00%,3,0,0,0,3,3,0
3,new,3,3,0,100.00%,3,0,0,0,3,3,0
12,waiter,3,2,1,66.67%,2,0,0,1,3,2,1
16,seafood,3,3,0,100.00%,2,1,0,0,3,2,1
25,pasta,3,2,1,66.67%,2,0,1,0,3,3,0


In [158]:
# total accuracy

sum_sent1 = sum(total_sent1)
sum_sent2 = sum(total_sent2)

sum_T1 = sum(TRUE_polar_ans1)
sum_T2 = sum(TRUE_polar_ans2)

print('Accuracy by target key (total_sent >= 1):')
print('rest1: ' + str(sum_T1) + '/' + str(sum_sent1) + ' = ' "%.2f%%" % (100 * sum_T1/sum_sent1))
print('rest2: ' + str(sum_T2) + '/' + str(sum_sent2) + ' = ' "%.2f%%" % (100 * sum_T2/sum_sent2))
print('total: ' + str(sum_T1+sum_T2) + '/' + str(sum_sent1+sum_sent2) + ' = ' "%.2f%%" % (100 * (sum_T1+sum_T2)/(sum_sent1+sum_sent2)))

print('-----------------------')

print('Accuracy by target key (total_sent >= 5):')
print('rest1: ' + str(34) + '/' + str(37) + ' = ' "%.2f%%" % (100 * 96/117))
print('rest2: ' + str(41) + '/' + str(52) + ' = ' "%.2f%%" % (100 * 71/79))
print('total: ' + str(34+41) + '/' + str(37+52) + ' = ' "%.2f%%" % (100 * (34+41)/(37+52)))

Accuracy by target key (total_sent >= 1):
rest1: 109/132 = 82.58%
rest2: 126/144 = 87.50%
total: 235/276 = 85.14%
-----------------------
Accuracy by target key (total_sent >= 5):
rest1: 34/37 = 82.05%
rest2: 41/52 = 89.87%
total: 75/89 = 84.27%
